In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pydrake.solvers.mathematicalprogram as mp

In [3]:
import numpy as np
import mpc_tools as mpc
import mpc_tools.mpcqp as mqp

In [33]:
m = 1.
l = 1.
g = 10.
A = np.array([
    [0., 1.],
    [g/l, 0.]
])
B = np.array([
    [0.],
    [1/(m*l**2.)]
])
t_s = .1
sys = mpc.DTLinearSystem.from_continuous(t_s, A, B)
N = 5
Q = np.eye(A.shape[0])/100.
R = np.eye(B.shape[1])
x_max = np.array([[np.pi/6.], [np.pi/20./(N*t_s)]])
x_min = -x_max
u_max = np.array([[m*g*l*np.pi/8.]])
u_min = -u_max

In [34]:
factory = mqp.MPCQPFactory(sys, N, Q, R)
factory.add_state_bound(x_max, x_min)
factory.add_input_bound(u_max, u_min)

In [35]:
mpc_qp = factory.assemble()

In [36]:
prog = mp.MathematicalProgram()
x = prog.NewContinuousVariables(2, N, "x")
u = prog.NewContinuousVariables(1, N, "u")
for j in range(N - 1):
    x_next = sys.A.dot(x[:, j]) + sys.B.dot(u[:, j])
    for i in range(x.shape[0]):
        prog.AddLinearConstraint(x[i, j + 1] == x_next[i])

for j in range(N):
    for i in range(x.shape[0]):
        prog.AddLinearConstraint(x[i, j] <= x_max[i])
        prog.AddLinearConstraint(x[i, j] >= x_min[i])
    for i in range(u.shape[0]):
        prog.AddLinearConstraint(u[i, j] <= u_max[i])
        prog.AddLinearConstraint(u[i, j] >= u_min[i])

for j in range(N):
    prog.AddQuadraticCost(x[:, j].dot(Q).dot(x[:, j]))
    prog.AddQuadraticCost(u[:, j].dot(R).dot(u[:, j]))

In [37]:
sym_qp = mqp.CanonicalMPCQP.from_mathematicalprogram(prog, u, x)

In [38]:
controller = mpc.MPCController(sym_qp, u.shape[0])

In [39]:
controller.compute_explicit_solution()

Computing critical region for the active set []


IndexError: too many indices for array